### General Remark : Since we work at the post level - we always compute an average on all comments per posts

In [2]:
import pandas as pd
import os
import spacy 
nlp = spacy.load("en_core_web_sm")
from tqdm import tqdm
from collections import defaultdict 
#Import des tokenizers - sent_tokenize pour les phrases et word_tokenize pour les mots avec nltk
from nltk.tokenize import sent_tokenize, word_tokenize
#On commence par importer la BD Wordnet depuis NLTK
from nltk.corpus import wordnet
import statistics
import numpy as np
from numpy.linalg import norm
import re
from tqdm import tqdm

import transformers
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import BertTokenizer, BertForNextSentencePrediction
from transformers import AutoTokenizer

from scipy.special import softmax

import torch

#Import Perso libraries
from ForIndivComments_Cognitive_AttentionAllocation import attention
from ForIndivComments_Cognitive_ActionComprehension import comprehension
from ForIndivComments_Cognitive_PriorExperience import experience
from ForIndivComments_Cognitive_EvalPersonAction import evaluation
from ForIndivComments_Cognitive_AnticipObserv import anticipation
from ForIndivComments_Cognitive_Relations import construction
from ForIndivComments_Affective import affect_relation
from ForIndivComments_Behavioral import behavior

import warnings
warnings.filterwarnings('ignore')

In [4]:
torch.cuda.is_available()

True

In [ ]:
# Loading dataset from influencers
#df_train = pd.read_csv("data/Train_Post_StoryPara_V1.csv")
#df_posts_train = df_train.dropna(subset=['PostText'])
#df_valid = pd.read_csv("data/Valid_Post_StoryPara_V1.csv")
#df_posts_valid = df_valid.dropna(subset=['PostText'])
#df_test = pd.read_csv("data/Test_Post_StoryPara_V1.csv")
#df_posts_test = df_test.dropna(subset=['PostText'])


#Loading comments from users
#df_train_c = pd.read_csv("data/Train_Comments_StoryPara_V1.csv")
#df_comments_train = df_train_c.dropna(subset=['CommentText'])
#df_comments_train['PostSysID'] = df_comments_train['PostSysID'].astype(int) 
#df_valid_c = pd.read_csv("data/Valid_Comments_StoryPara_V1.csv")
#df_comments_valid = df_valid_c.dropna(subset=['CommentText'])
#df_comments_valid['PostSysID'] = df_comments_valid['PostSysID'].astype(int) 
#df_test_c = pd.read_csv("data/Test_Comments_StoryPara_V1.csv")
#df_comments_test = df_test_c.dropna(subset=['CommentText'])
#df_comments_test['PostSysID'] = df_comments_test['PostSysID'].astype(int) 

In [ ]:
########################### GEt IDs lists
train_id = list(df_posts_train["PostSysID"])
valid_id = list(df_posts_valid["PostSysID"])
test_id = list(df_posts_test["PostSysID"])

In [8]:
#Function to pair to return lists of paired texts between Influencers and comments
def pairing(post_id, df, df_comments):
    
    df_post_ID = df[df["PostSysID"] == post_id]
    df_comment_ID = df_comments[df_comments["PostSysID"] == post_id]
    assert len(df_post_ID) != 0
    post_text = df_post_ID["PostText"].iloc[0]
    comments_list = list(df_comment_ID["CommentText"])
    
    return post_text, comments_list

# Part 1 - Cognitive Answer

## Attention Allocation

In [8]:
def compute_attention(id_list, df_post, df_comments):
    
    att_func = attention()
    for i in tqdm(range(len(id_list))):
        post_text, comments_list = pairing(id_list[i], df_post, df_comments)
        avg_min_all_com, avg_avg_all_com = att_func.attentive_vb(post_text, comments_list)
        count_user_list = att_func.follow_previous(id_list[i], df_post, df_comments)
        implication_list = att_func.post_implied(post_text, comments_list)

        df_comments.loc[df_comments['PostSysID']==id_list[i], 'vb_avg_min']= avg_min_all_com
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'vb_avg_avg']= avg_avg_all_com
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'avg_previous']= count_user_list
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'implication_list']= implication_list
    
    return df_comments

In [ ]:
#DONE - FULL - FULL
#df_new_valid = compute_attention(valid_id, df_posts_valid, df_comments_valid)
#df_new_valid.to_csv(f'data/NEW_Valid_Comments_StoryPara_WithVariables.csv', index=False)

#df_new_test = compute_attention(test_id, df_posts_test, df_comments_test)
#df_new_test.to_csv(f'data/NEW_Test_Comments_StoryPara_withVariable.csv', index=False)

## Comprehension of actions and situation

In [10]:
def compute_comprehension(id_list, df_post, df_comments):
    
    comp_func = comprehension()
    for i in tqdm(range(len(id_list))):
        post_text, comments_list = pairing(id_list[i], df_post, df_comments)
        list_semantic = comp_func.semantic_sim(post_text, comments_list)
        list_repeated = comp_func.repeated_kw(post_text, comments_list)
        list_predicted = comp_func.comment_predicted(post_text, comments_list)
        
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'semantic_similarity']= list_semantic
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'avg_repeat']= list_repeated
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'comment_predicted']= list_predicted
    
    return df_comments

In [12]:
#df_new_valid = compute_comprehension(valid_id, df_posts_valid, df_comments_valid)
#df_new_valid.to_csv(f'data/NEW_Valid_Comments_StoryPara_WithVariables2.csv', index=False)

#df_new_test = compute_comprehension(test_id, df_posts_test, df_comments_test)
#df_new_test.to_csv(f'data/NEW_Test_Comments_StoryPara_withVariables2.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████| 3920/3920 [16:10:23<00:00, 14.85s/it]


## Activation of prior experience

In [8]:
def compute_priorexp(id_list, df_post, df_comments):
    
    exp_func = experience()
    for i in tqdm(range(len(id_list))):
        post_text, comments_list = pairing(id_list[i], df_post, df_comments)
        #list_remember = exp_func.named_entities(comments_list)
        #list_imply = exp_func.causal_subord(comments_list)
        list_compare = exp_func.EN_compare(comments_list)

        #df_comments.loc[df_comments['PostSysID']==id_list[i], 'past_exp']= list_remember
        #df_comments.loc[df_comments['PostSysID']==id_list[i], 'impliedsituation']= list_imply
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'comparison']= list_compare
        
    return df_comments

In [10]:
#DONE - FULL - FULL
df_new_valid = compute_priorexp(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/NEW_Valid_Comments_StoryPara_WithVariables2.csv', index=False)

df_new_test = compute_priorexp(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/NEW_Test_Comments_StoryPara_withVariables3.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████| 3920/3920 [35:42<00:00,  1.83it/s]


## Evaluation of Persona's actions

In [8]:
### Faire le for loop ici avec le calc_score et la méthode method="m-1"/method="m-2" for subjectivity and neutrality
def compute_evalperso(id_list, df_post, df_comments):
    
    eval_func = evaluation()

    for i in tqdm(range(len(id_list))):
        post_text, comments_list = pairing(id_list[i], df_post, df_comments)
        #list_weak_subj, list_strong_subj = eval_func.subj_eval(comments_list)
        list_neutral, list_objective = eval_func.calc_scores(comments_list, method="m-2")

        #df_comments.loc[df_comments['PostSysID']==id_list[i], 'weak_subj']= list_weak_subj
       # df_comments.loc[df_comments['PostSysID']==id_list[i], 'strong_subj']= list_strong_subj
        #df_comments.loc[df_comments['PostSysID']==id_list[i], 'neutral']= list_neutral
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'objective']= list_objective
    
    return df_comments

In [10]:
############################################################################################
#DONE - FULL - FULL
#df_new_valid = compute_evalperso(valid_id, df_posts_valid, df_comments_valid)
#df_new_valid.to_csv(f'data/NEW_Valid_Comments_StoryPara_WithVariables2.csv', index=False)

#df_new_test = compute_evalperso(test_id, df_posts_test, df_comments_test)
#df_new_test.to_csv(f'data/NEW_Test_Comments_StoryPara_withVariables3.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████| 3920/3920 [13:24<00:00,  4.87it/s]


## Anticipatory observation

In [14]:
#For using those function you might process df_post to take user username list with the post and the comments
def compute_anticipobser(id_list, df_post, df_comments):
    
    anticip_func = anticipation()
    for i in tqdm(range(len(id_list))):
        
        post_text, comments_list = pairing(id_list[i], df_post, df_comments)
        username = df_post[df_post['PostSysID'] == id_list[i]]['PostUser'].iloc[0]
        list_brand_evo = anticip_func.brand_evo(username, comments_list)
        list_brand_condi = anticip_func.brand_cond(username, comments_list)

        df_comments.loc[df_comments['PostSysID']==id_list[i], 'brand_evolution']= list_brand_evo
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'brand_conditional']= list_brand_condi
            
    return df_comments

In [16]:
#DONE - FULL - FULL
#df_new_valid = compute_anticipobser(valid_id, df_posts_valid, df_comments_valid)
#df_new_valid.to_csv(f'data/NEW_Valid_Comments_StoryPara_WithVariables2.csv', index=False)

#df_new_test = compute_anticipobser(test_id, df_posts_test, df_comments_test)
#df_new_test.to_csv(f'data/NEW_Test_Comments_StoryPara_withVariables3.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████| 3920/3920 [1:05:12<00:00,  1.00it/s]


## Relation construction

In [20]:
#For using those function you might process df_post to take user username list with the post and the comments
def compute_relconstru(id_list, df_post, df_comments):
    
    construc_func = construction()
    for i in tqdm(range(len(id_list))):
        
        post_text, comments_list = pairing(id_list[i], df_post, df_comments)
        username = df_post[df_post['PostSysID'] == id_list[i]]['PostUser'].iloc[0]
        list_context = construc_func.context_proj(username, comments_list)
        list_obsession = construc_func.obsession(username, comments_list)
        list_action = construc_func.action_link(username, comments_list)
        list_mention = construc_func.direct_mention(username, comments_list)

        df_comments.loc[df_comments['PostSysID']==id_list[i], 'contextual_projection']= list_context
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'obsession']= list_obsession
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'link_my_action']= list_action
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'direct_mention']= list_mention

    return df_comments

In [22]:
#DONE - FULL  - FULL
#df_new_valid = compute_relconstru(valid_id, df_posts_valid, df_comments_valid)
#df_new_valid.to_csv(f'data/NEW_Valid_Comments_StoryPara_WithVariables2.csv', index=False)

#df_new_test = compute_relconstru(test_id, df_posts_test, df_comments_test)
#df_new_test.to_csv(f'data/NEW_Test_Comments_StoryPara_withVariables3.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████| 3920/3920 [1:02:09<00:00,  1.05it/s]


# Part 2 - Affective

## Sympathy / Empathy / Sentiment contagion

In [10]:
#For using those function you might process df_post to take user username list with the post and the comments
def compute_affect(id_list, df_post, df_comments):
    
    affect_func = affect_relation()

    for i in tqdm(range(len(id_list))):
        post_text, comments_list = pairing(id_list[i], df_post, df_comments)
        list_sentiment, list_intense = affect_func.sentiment(comments_list, feel_threshold=0.8)
        list_empathy, list_same_intens = affect_func.empathy(post_text, comments_list, sim_intens=0.1)
        list_contagion = affect_func.contagion(post_text, comments_list)
        list_letterrepeat = affect_func.nb_letter_repeat(comments_list)
        list_letterupper = affect_func.prop_upper(comments_list)

        df_comments.loc[df_comments['PostSysID']==id_list[i], 'sentiment']= list_sentiment
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'intensity']= list_intense
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'empathy']= list_empathy
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'same_intensity']= list_same_intens
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'sentiment_contagion']= list_contagion
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'nb_letter_repeat']= list_letterrepeat
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'nb_uppercase']= list_letterupper

    return df_comments

In [12]:
#DONE - FULL - FULL
#df_new_valid = compute_affect(valid_id, df_posts_valid, df_comments_valid)
#df_new_valid.to_csv(f'data/NEW_Valid_Comments_StoryPara_WithVariables2.csv', index=False)

#df_new_test = compute_affect(test_id, df_posts_test, df_comments_test)
#df_new_test.to_csv(f'data/NEW_Test_Comments_StoryPara_withVariables3.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████| 3920/3920 [21:09:01<00:00, 19.42s/it]


# Part 3 - Behavioral

## Non verbal / Para verbal / intentions

In [8]:
#For using those function you might process df_post to take user username list with the post and the comments
def compute_behavior(id_list, df_post, df_comments):
    
    behav_func = behavior()
    for i in tqdm(range(len(id_list))):
        post_text, comments_list = pairing(id_list[i], df_post, df_comments)
        username = df_post[df_post['PostSysID'] == id_list[i]]['PostUser'].iloc[0]
        
        list_emoji, list_nb_emoji = behav_func.behav_emoji(comments_list)
        list_nb_comments = behav_func.nb_comments(comments_list)
        list_slangs = behav_func.behav_slangs(comments_list)
        list_mistakes, list_grammar = behav_func.error_type(comments_list)
        list_time_to_write = behav_func.take_time_to_write(comments_list, username, nb_words = 20, nb_charac=80)
        list_answer = behav_func.answer(comments_list, username)
        list_intent = behav_func.intent(comments_list)

        df_comments.loc[df_comments['PostSysID']==id_list[i], 'emojis']= list_emoji
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'nb_comments_emojis']= list_nb_emoji
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'nb_comments']= list_nb_comments
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'nb_slangs']= list_slangs
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'nb_mistakes']= list_mistakes
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'nb_grammar_errors']= list_grammar
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'time_to_write']= list_time_to_write
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'nb_answers']= list_answer
        df_comments.loc[df_comments['PostSysID']==id_list[i], 'intent']= list_intent
        
    return df_comments

In [10]:

#DONE - FULL - FULL
df_new_valid = compute_behavior(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/NEW_Valid_Comments_StoryPara_WithVariables2.csv', index=False)

df_new_test = compute_behavior(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/NEW_Test_Comments_StoryPara_withVariables3.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████| 3920/3920 [3:51:44<00:00,  3.55s/it]


# Computing everything

In [ ]:
#DONE - FULL - FULL
df_new_valid = compute_attention(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/Valid_Parasocial_att_withVariable.csv', index=False)

df_new_test = compute_attention(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/Test_Parasocial_att_withVariable.csv', index=False)

In [ ]:
#DONE - FULL - FULL
df_new_valid = compute_comprehension(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/Valid_Parasocial_comp_withVariable.csv', index=False)

df_new_test = compute_comprehension(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/Test_Parasocial_comp_withVariable.csv', index=False)

In [14]:
#DONE - FULL - FULL
df_new_valid = compute_priorexp(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/Valid_Parasocial_prior_withVariable.csv', index=False)

df_new_test = compute_priorexp(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/Test_Parasocial_prior_withVariable.csv', index=False)

In [13]:
#DONE - FULL - FULL
df_new_valid = compute_evalperso(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/Valid_Parasocial_eval_withVariable.csv', index=False)

df_new_test = compute_evalperso(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/Test_Parasocial_eval_withVariable.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████| 3920/3920 [3:29:11<00:00,  3.20s/it]


In [13]:
#DONE - FULL - FULL
df_new_valid = compute_anticipobser(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/Valid_Parasocial_anticip_withVariable.csv', index=False)

df_new_test = compute_anticipobser(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/Test_Parasocial_anticip_withVariable.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████| 3920/3920 [2:02:10<00:00,  1.87s/it]


In [14]:
#DONE - FULL  - FULL
df_new_valid = compute_relconstru(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/Valid_Parasocial_rel_withVariable.csv', index=False)

df_new_test = compute_relconstru(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/Test_Parasocial_rel_withVariable.csv', index=False)

In [13]:
#DONE - FULL - FULL
df_new_valid = compute_affect(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/Valid_Parasocial_affect_withVariable.csv', index=False)

df_new_test = compute_affect(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/Test_Parasocial_affect_withVariable.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████| 3920/3920 [21:15:45<00:00, 19.53s/it]


In [13]:
#DONE - FULL - FULL
df_new_valid = compute_behavior(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/Valid_Parasocial_behav_withVariable.csv', index=False)

df_new_test = compute_behavior(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/Test_Parasocial_behav_withVariable.csv', index=False)

# Recomputing the ones that did not work

In [9]:
#def compute_fail(id_list, df_post, df_comments):
#    
#    behav_func = behavior()
#    eval_func = evaluation()
#    comp_func = comprehension()
#    
#    list_emoji, list_nb_emoji, list_repeated, list_neutral, list_objective = [], [], [], [], []
#    for i in tqdm(range(len(id_list))):
#        post_text, comments_list = pairing(id_list[i], df_post, df_comments)
#        username = df_post[df_post['PostSysID'] == id_list[i]]['PostUser'].iloc[0]
#        
#        avg_emojis, nb_comment_emoji = behav_func.behav_emoji(comments_list)
#        neutral, objective = eval_func.calc_scores(comments_list, method="m-1")
#        avg_repeat = comp_func.repeated_kw(post_text, comments_list)
#        
#        list_emoji.append(avg_emojis)
#        list_nb_emoji.append(nb_comment_emoji)
#        list_repeated.append(avg_repeat)
#        list_neutral.append(neutral)
#        list_objective.append(objective)
#        
#    df_post["emojis"] = list_emoji
#    df_post["nb_comments_emojis"] = nb_comment_emoji
#    df_post["neutral"] = list_neutral
#    df_post["objectivity"] = list_objective
#    df_post["avg_repeat"] = list_repeated
#    
#    return df_post

In [10]:
df_new_valid = compute_fail(valid_id, df_posts_valid, df_comments_valid)
df_new_valid.to_csv(f'data/Valid_Parasocial_failed_withVariable.csv', index=False)

df_new_test = compute_fail(test_id, df_posts_test, df_comments_test)
df_new_test.to_csv(f'data/Test_Parasocial_failed_withVariable.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████| 3920/3920 [09:37<00:00,  6.79it/s]


## Nouvelles variables après correction

In [12]:
def engage_rate(post_id, df_post, df_comments):
        
        #GEtting the username and the date linked to the post
        temp = df_post[df_post["PostSysID"] == post_id]
        user_name = temp["PostUser"].iloc[0]
        post_date = temp["PostDate"].iloc[0]

        #Getting the dataframe of all posts associated to the user
        df_user = df_post[df_post["PostUser"] == user_name]
        #Getting post_id list of previous posts of that user and post afters
        post_id_prev = df_user[df_user["PostDate"] < post_date]["PostSysID"]
        post_id_after = df_user[df_user["PostDate"] >= post_date]["PostSysID"]
        
        #GEtting the current commenters list
        commenter_list = list(df_comments[df_comments["PostSysID"] == post_id]["CommentUser"])
        
        #Collecting comments associated to previous and post after the current post : 
        df_comments_prev = df_comments[df_comments["PostSysID"].isin(post_id_prev)]
        prev_commenter_list = list(df_comments_prev["CommentUser"])
        
        df_comments_after = df_comments[df_comments["PostSysID"].isin(post_id_after)]
        after_commenter_list = list(df_comments_after["CommentUser"])
        
        avg_user_count = []
        for user in commenter_list:
            count_prev = prev_commenter_list.count(user)
            count_after = after_commenter_list.count(user)
            engage_ratio = count_after / (count_prev + 1)
            avg_user_count.append(engage_ratio)

        #On average user have commented n previous posts of the same instagram influencer
        return avg_user_count

In [16]:
# POUR VALID -- 
###############ATTENTION -- RELOAD FILE WITH THE COLUMN ALREADY SAVE -- COMMENTS WITH THE NEW VARIABLES
df_train = pd.read_csv("data/Train_Post_StoryPara_V1.csv")
df_posts_train = df_train.dropna(subset=['PostText'])
df_valid = pd.read_csv("data/Valid_Post_StoryPara_V1.csv")
df_posts_valid = df_valid.dropna(subset=['PostText'])
df_test = pd.read_csv("data/Test_Post_StoryPara_V1.csv")
df_posts_test = df_test.dropna(subset=['PostText'])

df_valid_c = pd.read_csv("data/NEW_Valid_Comments_StoryPara_WithVariables2.csv")
df_comments_valid = df_valid_c.dropna(subset=['CommentText'])
df_comments_valid['PostSysID'] = df_comments_valid['PostSysID'].astype(int) 
df_test_c = pd.read_csv("data/NEW_Test_Comments_StoryPara_withVariables3.csv")
df_comments_test = df_test_c.dropna(subset=['CommentText'])
df_comments_test['PostSysID'] = df_comments_test['PostSysID'].astype(int)
valid_id = list(df_posts_valid["PostSysID"])
test_id = list(df_posts_test["PostSysID"])

#for i in tqdm(range(len(valid_id))):
 #   post_text, comments_list = pairing(valid_id[i], df_posts_valid, df_comments_valid)
#    username = df_posts_valid[df_posts_valid['PostSysID'] == valid_id[i]]['PostUser'].iloc[0]
#    list_enage = engage_rate(valid_id[i], df_posts_valid, df_comments_valid)
    
#    df_comments_valid.loc[df_comments_valid['PostSysID']==valid_id[i], 'engage_rate']= list_enage

#df_comments_valid.to_csv(f'data/NEW_Valid_Comments_StoryPara_WithVariables2.csv', index=False)

for i in tqdm(range(len(test_id))):
    post_text, comments_list = pairing(test_id[i], df_posts_test, df_comments_test)
    username = df_posts_test[df_posts_test['PostSysID'] == test_id[i]]['PostUser'].iloc[0]
    list_enage = engage_rate(test_id[i], df_posts_test, df_comments_test)
    
    df_comments_test.loc[df_comments_test['PostSysID']==test_id[i], 'engage_rate']= list_enage

100%|██████████████████████████████████████████████████████████████████████████████| 3920/3920 [06:47<00:00,  9.62it/s]


In [11]:
## POUR TEST -- 
##relancer pour corriger ce bug
#test_id = list(df_posts_test["PostSysID"])
#
#exp_func = experience()
#comp_func = comprehension()
#behav_func = behavior()
#
#list_enage, list_imply, list_time_to_write, list_repeat, list_comment_emoji = [], [], [], [], []
#for i in tqdm(range(len(test_id))):
#    post_text, comments_list = pairing(test_id[i], df_posts_test, df_comments_test)
#    username = df_posts_test[df_posts_test['PostSysID'] == test_id[i]]['PostUser'].iloc[0]
#    
#    avg_engage = engage_rate(test_id[i], df_posts_test, df_comments_test)
#    avg_imply = exp_func.causal_subord(comments_list)
#    avg_repeat = comp_func.repeated_kw(post_text, comments_list)
#    avg_time_to_write = behav_func.take_time_to_write(comments_list, username, nb_words = 20, nb_charac=80)
#    _, nb_comment_emoji = behav_func.behav_emoji(comments_list)
#    
#    list_enage.append(avg_engage)
#    list_imply.append(avg_imply)
#    list_time_to_write.append(avg_time_to_write)
#    list_repeat.append(avg_repeat)
#    list_comment_emoji.append(nb_comment_emoji)
#    
#df_posts_test["nb_comments_emojis"] = list_comment_emoji
#df_posts_test["engage_rate"] = list_enage
#df_posts_test["comments_implied"] = list_imply
#df_posts_test["avg_repeat"] = list_repeat
#df_posts_test["time_to_write"] = list_time_to_write
#
#df_posts_test.to_csv(f'data/Test_Parasocial_Corrected.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████| 3920/3920 [09:25<00:00,  6.93it/s]
